In [22]:
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cf
import numpy as np
import xarray as xr
import time
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES)
import xesmf as xe
from xgcm.autogenerate import generate_grid_ds
from xgcm import Grid
from scipy import stats

In [74]:
ll = xr.open_dataset("../NOV02_forecast/data/geo_em.d01.nc")
grid_in = xr.Dataset(
    {
        "lat": (["y", "x"],ll['XLAT_M'].squeeze().values, {"units": "degrees_north"}),
        "lon": (["y", "x"], ll['XLONG_M'].squeeze().values, {"units": "degrees_east"}),
        "lat_b": (["y_b", "x_b"], ll['XLAT_C'].squeeze().values, {"units": "degrees_north"}),
        "lon_b": (["y_b", "x_b"], ll['XLONG_C'].squeeze().values, {"units": "degrees_east"}),
    }
)
grid_in

<xarray.Dataset>
Dimensions:  (y: 671, x: 767, y_b: 672, x_b: 768)
Dimensions without coordinates: y, x, y_b, x_b
Data variables:
    lat      (y, x) float32 9.692 9.695 9.698 9.701 ... 33.58 33.57 33.57 33.57
    lon      (y, x) float32 31.28 31.32 31.35 31.39 ... 61.14 61.18 61.22 61.26
    lat_b    (y_b, x_b) float32 9.672 9.676 9.679 9.682 ... 33.59 33.59 33.58
    lon_b    (y_b, x_b) float32 31.27 31.3 31.34 31.37 ... 61.2 61.24 61.29

In [88]:

filename = "../NOV02_forecast/data/Daily.Rain.2023-11-02.nc"
ds = xr.open_dataset(filename)

lat_out = np.arange(9.69, 34.291, 0.04)
lat_out_b = np.arange(9.67, 34.311, 0.04)
lon_out = np.arange(28.72, 61.281, 0.04)
lon_out_b = np.arange(28.7, 61.31, 0.04)
grid_out = {
    "lon": lon_out,
    "lat": lat_out,
    "lon_b": lon_out_b,
    "lat_b": lat_out_b,
}
#grid_out


In [83]:
regridder = xe.Regridder(grid_in, grid_out, method='conservative')
regridder

xESMF Regridder 
Regridding algorithm:       conservative 
Weight filename:            conservative_671x767_616x815.nc 
Reuse pre-computed weights? False 
Input grid shape:           (671, 767) 
Output grid shape:          (616, 815) 
Periodic in longitude?      False

In [89]:
rainnc_out = regridder(ds['RAINNC'], keep_attrs=True)
rainnc_out.to_netcdf('rain_out.nc')

In [90]:
filename = "../NOV02_forecast/data/Daily.Temp.2023-11-02.nc"
ds = xr.open_dataset(filename)
temp_out = regridder(ds['T2'], keep_attrs=True)
temp_out.to_netcdf('temp_out.nc')